In [1]:
import numpy as np
import scipy.misc
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import keras

import math

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
imageNames = os.listdir("./dataset/")

for imageName in imageNames:
    print(imageName)

IMG_20151119_150243.jpg
IMG_20151120_160602.jpg
IMG_20151120_161800.jpg
IMG_20151120_162301.jpg
IMG_20151120_162331.jpg
IMG_20151120_164001.jpg
IMG_20151120_164015.jpg
IMG_20151121_160323.jpg
IMG_20151121_160330.jpg
IMG_20151122_151323.jpg
IMG_20151122_151412.jpg
IMG_20151123_144208.jpg
IMG_20151123_144852.jpg
IMG_20151123_145007.jpg
IMG_20151123_145326.jpg
IMG_20151123_145752.jpg
IMG_20151123_150812.jpg
IMG_20151123_154943.jpg
IMG_20151123_161018.jpg
IMG_20151123_161937.jpg
IMG_20151123_162549.jpg
IMG_20151123_162551.jpg
IMG_20151123_163354.jpg
IMG_20151123_163606.jpg
IMG_20151123_164230.jpg
IMG_20160128_132134.jpg
IMG_20160128_132139.jpg
IMG_20160325_112547.jpg
IMG_20160325_113508.jpg
IMG_20170913_154424.jpg


In [3]:
def squareify(img):
    r = img.shape[0]
    c = img.shape[1]
    if r == c:
        return img
    elif r > c:
        top = (r - c) // 2
        bottom = top + c
        return img[top:bottom, :]
    else:
        left = (c - r) // 2
        right = left + r
        return img[:, left:right]

In [4]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.587, 0.299, 0.114])

def sharpen(img, sigma, sharp_weight):
    # A high pass filter created by subtracting the low pass from the original
    low_pass_img = scipy.ndimage.filters.gaussian_filter(img, sigma)
    high_pass_img = rgb2gray(img - low_pass_img)
    high_pass_img = np.dstack([high_pass_img, high_pass_img, high_pass_img])
    
    result = (sharp_weight * high_pass_img + img)
    
    def limit(x):
        if x < 0:
            return 0
        elif x > 1:
            return 1
        return x
    
    clamp = np.vectorize(limit)
    
    result = clamp(result)
    
    return result

In [5]:
def gaussianStack(image, num_layers, sigma):
    results = []
    for i in range(num_layers):
        if i == 0:
            results.append(scipy.ndimage.filters.gaussian_filter(image, sigma))
        else:
            results.append(scipy.ndimage.filters.gaussian_filter(results[i-1], sigma))
    return results

def laplacianStack(image, num_layers, sigma):
    results = []
    for i in range(num_layers):
        if i == 0:
            results.append(scipy.ndimage.filters.laplace(image))
        else:
            results.append(scipy.ndimage.filters.gaussian_filter(results[i-1], sigma))
    return results

In [6]:
baseImages = {}

for imageName in imageNames:
    img = scipy.misc.imread("./dataset/" + imageName)
    img = squareify(img)
    img = scipy.misc.imresize(img, (1024, 1024, 3))
    baseImages[imageName] = rgb2gray(img) # img[:, :, 0]
    
    scipy.misc.imsave("./dataset_rect/" + imageName, img)

c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.
c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if __name__ == '__main__':


In [7]:
for imageName in imageNames:
    img = scipy.misc.imread("./dataset/" + imageName)
    
    gs = gaussianStack(img, 6, 3.0)
    ls = laplacianStack(img, 6, 3.0)
    
    for i in range(len(gs)):
        gaussian = gs[i]
        scipy.misc.imsave("./dataset_gaussian/" + imageName.split(".")[0] + "_gaussian_" + str(i) + ".jpg", gaussian)
    for i in range(len(ls)):
        laplacian = ls[i]
        scipy.misc.imsave("./dataset_laplacian/" + imageName.split(".")[0] + "_laplacian_" + str(i) + ".jpg", laplacian)
    bw = rgb2gray(img)
    scipy.misc.imsave("./dataset_bw/" + imageName.split(".")[0] + "_bw_" + str(i) + ".jpg", bw)

c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if __name__ == '__main__':
c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':
c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [8]:
class Generator:
    def __init__(self, depths=[1024, 512, 256, 128], s_size=4):
        self.depths = depths + [3]
        self.s_size = s_size
        self.reuse = False

    def __call__(self, inputs, training=False):
        inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            with tf.variable_scope('reshape'):
                outputs = tf.layers.dense(inputs, self.depths[0] * self.s_size * self.s_size)
                outputs = tf.reshape(outputs, [-1, self.s_size, self.s_size, self.depths[0]])
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            # deconvolution (transpose of convolution) x 4
            with tf.variable_scope('deconv1'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv2'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv3'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv4'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
            # output images
            with tf.variable_scope('tanh'):
                outputs = tf.tanh(outputs, name='outputs')
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs


class Discriminator:
    def __init__(self, depths=[64, 128, 256, 512]):
        self.depths = [3] + depths
        self.reuse = False

    def __call__(self, inputs, training=False, name=''):
        def leaky_relu(x, leak=0.2, name=''):
            return tf.maximum(x, x * leak, name=name)
        outputs = tf.convert_to_tensor(inputs)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            # convolution x 4
            with tf.variable_scope('conv1'):
                outputs = tf.layers.conv2d(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv2'):
                outputs = tf.layers.conv2d(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv3'):
                outputs = tf.layers.conv2d(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv4'):
                outputs = tf.layers.conv2d(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('classify'):
                batch_size = outputs.get_shape()[0].value
                reshape = tf.reshape(outputs, [batch_size, -1])
                outputs = tf.layers.dense(reshape, 2, name='outputs')
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs


class DCGAN:
    def __init__(self,
                 batch_size=8, s_size=4, z_dim=100,
                 g_depths=[1024, 512, 256, 128],
                 d_depths=[64, 128, 256, 512]):
        self.batch_size = batch_size
        self.s_size = s_size
        self.z_dim = z_dim
        self.g = Generator(depths=g_depths, s_size=self.s_size)
        self.d = Discriminator(depths=d_depths)
        self.z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0)

    def loss(self, traindata):
        """build models, calculate losses.
        Args:
            traindata: 4-D Tensor of shape `[batch, height, width, channels]`.
        Returns:
            dict of each models' losses.
        """
        generated = self.g(self.z, training=True)
        g_outputs = self.d(generated, training=True, name='g')
        t_outputs = self.d(traindata, training=True, name='t')
        # add each losses to collection
        tf.add_to_collection(
            'g_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.ones([self.batch_size], dtype=tf.int64),
                    logits=g_outputs)))
        tf.add_to_collection(
            'd_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.ones([self.batch_size], dtype=tf.int64),
                    logits=t_outputs)))
        tf.add_to_collection(
            'd_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.zeros([self.batch_size], dtype=tf.int64),
                    logits=g_outputs)))
        return {
            self.g: tf.add_n(tf.get_collection('g_losses'), name='total_g_loss'),
            self.d: tf.add_n(tf.get_collection('d_losses'), name='total_d_loss'),
        }

    def train(self, losses, learning_rate=0.0002, beta1=0.5):
        """
        Args:
            losses dict.
        Returns:
            train op.
        """
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
        g_opt_op = g_opt.minimize(losses[self.g], var_list=self.g.variables)
        d_opt_op = d_opt.minimize(losses[self.d], var_list=self.d.variables)
        with tf.control_dependencies([g_opt_op, d_opt_op]):
            return tf.no_op(name='train')

    def sample_images(self, row=8, col=8, inputs=None):
        if inputs is None:
            inputs = self.z
        images = self.g(inputs, training=True)
        images = tf.image.convert_image_dtype(tf.div(tf.add(images, 1.0), 2.0), tf.uint8)
        images = [image for image in tf.split(images, self.batch_size, axis=0)]
        rows = []
        for i in range(row):
            rows.append(tf.concat(images[col * i + 0:col * i + col], 2))
        image = tf.concat(rows, 1)
        
        return tf.image.encode_jpeg(tf.squeeze(image, [0]))

In [9]:
imagesLoaded = []

for imageName in imageNames:
    result = np.asarray(Image.open("./dataset_rect/" + imageName).convert('L'))
    imagesLoaded.append(result)
    
len(imagesLoaded)

30

In [10]:
imagesLoaded[0]

array([[244, 244, 244, ..., 231, 232, 233],
       [244, 244, 244, ..., 232, 233, 233],
       [244, 244, 244, ..., 234, 234, 235],
       ..., 
       [174, 173, 171, ...,  44,  44,  44],
       [160, 159, 158, ...,  44,  44,  44],
       [151, 151, 152, ...,  44,  44,  44]], dtype=uint8)

In [11]:
dcgan = DCGAN()
train_images = imagesLoaded
losses = dcgan.loss(imagesLoaded)
train_op = dcgan.train(losses)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(FLAGS.max_steps):
        _, g_loss_value, d_loss_value = sess.run([train_op, losses[dcgan.g], losses[dcgan.d]])

ValueError: Argument must be a dense tensor: [array([[244, 244, 244, ..., 231, 232, 233],
       [244, 244, 244, ..., 232, 233, 233],
       [244, 244, 244, ..., 234, 234, 235],
       ..., 
       [174, 173, 171, ...,  44,  44,  44],
       [160, 159, 158, ...,  44,  44,  44],
       [151, 151, 152, ...,  44,  44,  44]], dtype=uint8), array([[143, 123,  66, ..., 205, 205, 154],
       [143, 131,  82, ..., 231, 249, 182],
       [162, 155, 107, ..., 230, 244, 157],
       ..., 
       [ 46,  44,  42, ...,  41,  43,  45],
       [ 46,  46,  45, ...,  41,  41,  42],
       [ 41,  43,  45, ...,  44,  41,  39]], dtype=uint8), array([[154, 153, 153, ..., 130, 130, 130],
       [154, 153, 153, ..., 130, 130, 130],
       [154, 154, 153, ..., 130, 130, 130],
       ..., 
       [ 17,  14,  12, ...,  21,  22,  24],
       [ 15,  10,   9, ...,  20,  22,  24],
       [ 10,   5,   5, ...,  17,  20,  22]], dtype=uint8), array([[  8,  14,  18, ..., 186, 187, 188],
       [  4,   8,  12, ..., 187, 1